In [1]:
import yahoo_fin.stock_info as si
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from pandas import read_html
import ray
import time

nan=np.nan

In [2]:
@ray.remote
def GrahamInVal (sticker):
    #sticker = "CRM"

    # Read EPS
    url_Summary = 'https://finance.yahoo.com/quote/' + sticker
    dframe_list_Summary = pd.read_html(url_Summary)
    Summary_tab = dframe_list_Summary[1]
    EPS_str = Summary_tab.loc[3,1]
    EPS = nan
    if pd.isnull(EPS_str) == 0:
        if EPS_str.find('%'):
            EPS = float(EPS_str[:-1])
        else:
            EPS = float(EPS_str)
        
    # Read P/E
    #PERatio_str = Summary_tab.loc[2,1]
    #PERatio_read = float(PERatio_str)
    PERatio_fixed = 7.0
    PERatio = PERatio_fixed

    # Read 5 Yr Growth Rate
    url_Analysis = 'https://finance.yahoo.com/quote/' + sticker + '/analysis?p=' + sticker
    GrowthEst = nan
    try:
        dframe_list_Analysis = pd.read_html(url_Analysis)
        if dframe_list_Analysis!=dframe_list_Summary:
            GrowthEst_tab = dframe_list_Analysis[5]
            GrowthEst_str = GrowthEst_tab.loc[4,sticker]
            if pd.isnull(GrowthEst_str) == 0:
                GrowthEst = float(GrowthEst_str[:-1])
    except ValueError:
        pass
    GrowthScale = 1

    # Read AAACorpBond Yield
    url_AAACorpBond = 'https://ycharts.com/indicators/moodys_seasoned_aaa_corporate_bond_yield'
    dframe_list_AAACorpBond = pd.read_html(url_AAACorpBond)
    dframe_AAACorpBond = dframe_list_AAACorpBond[0]
    AAACorpBond_str = dframe_AAACorpBond.loc[0,'Unnamed: 1']
    AAACorpBond = nan
    if pd.isnull(AAACorpBond_str) == 0:
        AAACorpBond = float(AAACorpBond_str[:-1])
    CorpBond = 4.4

    IntrinsicVal = round((EPS * (PERatio + (GrowthEst * GrowthScale)) * CorpBond) / AAACorpBond,2)

    # Read Current Val
    Summary_tab = dframe_list_Summary[0]
    PrevClose_str = Summary_tab.loc[0,1]
    PrevClose = round(float(PrevClose_str),2)

    Decision = nan
    if pd.isnull(IntrinsicVal) == 0:
        if PrevClose < IntrinsicVal:
            Decision = 'BUY'
        else:
            Decision = 'SELL'

    MarginRaw = (IntrinsicVal-PrevClose)/PrevClose
    Margin = round(MarginRaw*100,2)
    #Margin = '{percent:.2%}'.format(percent=MarginRaw)
    
    time.sleep(1)

    return {'Sticker':sticker,
             'IntrinsicVal($)':IntrinsicVal,
             'PrevClose($)':PrevClose,
             'Decision':Decision,
             'Margin(%)':Margin,
             'EPS':EPS,
             'GrowthEst(%)':GrowthEst}

In [3]:
#DO NOT TOUCH THIS

#stock_val = GrahamInVal("AAPL")

# Create the pandas DataFrame 
#stock_tabsum = pd.DataFrame.from_dict(stock_val, orient='index')
#stock_sum = stock_tabsum.transpose()

# print dataframe. 
#stock_sum.set_index('Sticker', inplace=True)


In [ ]:
nasdaq_list = si.tickers_nasdaq()
#stock_list = nasdaq_list[:10]
#stock_list = nasdaq_list[:50]
stock_list = ['AAPL','SHOP','MA','PYPL','SQ','WIX']
print(stock_list)
   
stock_sum = pd.DataFrame(np.arange(7*len(stock_list)).reshape((len(stock_list),7)),
                         columns=['Sticker',
                                  'IntrinsicVal($)',
                                  'PrevClose($)',
                                  'Decision', 
                                  'Margin(%)',
                                  'EPS',
                                  'GrowthEst(%)'])

ray.init()

def function_y(stock_list):
    pass

# Process the items in parallel (see ray initialization in function)
results = ray.get([GrahamInVal.remote(sticky) for sticky in stock_list])

function_y(stock_list)

len_stock_list=np.arange(len(stock_list))
for i in len_stock_list:
    ami1 = pd.DataFrame.from_dict(results[i], orient='index')
    ami2 = ami1.transpose()
    stock_sum = stock_sum.append(ami2)

stock_sum.set_index('Sticker', inplace=True)
stock_sum.drop(stock_sum.index[0:len(stock_list)], inplace=True)
stock_sum #print stock_sum

2020-08-17 23:52:44,858	INFO resource_spec.py:212 -- Starting Ray with 0.29 GiB memory available for workers and up to 0.15 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


['AAPL', 'SHOP', 'MA', 'PYPL', 'SQ', 'WIX']


2020-08-17 23:52:45,743	INFO services.py:1165 -- View the Ray dashboard at localhost:8265
